# Atividade 1



In [1]:
# Bibliotecas utilizadas
import json
import secrets

import gurobipy as gp
from gurobipy import *

In [2]:
#
# Funções auxiliares
#

# Range inclusive
def inclrange(min, max):
    return range(min, max + 1)

# Inteiro aleatório entre min e max inclusive
def rand(min, max):
    return min + secrets.randbelow(max - min + 1)

# Salva um arquivo JSON com o dicionário fornecido
def imprimir_json(nome_arquivo, dados):
    with open(nome_arquivo, 'w', encoding='utf-8') as arquivo:
        json.dump(dados, arquivo, ensure_ascii = False, indent = 4)

#
# Configurações
#

limitar_threads = False # 😎 
imprimir_dados = True
imprimir_resultado = True


### Limites das variáveis das instâncias

```
|J| = {100, 200, 300, 400, 500, 600, 700, 800, 900, 1000}
|F| ∈ [|J|, 2|J|]
|L| ∈ [5, 10]
|M| ∈ [5, 10]
|P| ∈ [5, 10]
D_j,p ∈ [10, 20]
r_m,p,l ∈ [1, 5]
R_m,f ∈ [800, 1000]
C_l,f ∈ [80, 100]
p_p,l,f ∈ [10, 100]
t_p,f,j ∈ [10, 20]
```

In [3]:
#
# Executar solução
#

def solucionar(J, F, L, M, P, D, r, R, C, cp, ct):
    # Criação do modelo
    modelo = gp.Model(f'Atividade 1 - {len(J)} clientes')
    if limitar_threads:
        modelo.setParam(GRB.Param.Method, 1)
        modelo.setParam(GRB.Param.Threads, 1)

    # Variáveis de decisão
    tonelada_transportada = modelo.addVars(P, F, J, lb = 0, vtype = GRB.CONTINUOUS)
    tonelada_produzida = modelo.addVars(P, L, F, lb = 0, vtype = GRB.CONTINUOUS)
    custo = modelo.addVar(lb = 0, vtype = GRB.CONTINUOUS)
    
    # Função objetivo
    obj = quicksum(quicksum(quicksum(tonelada_produzida.sum(p,l,f) * cp[p][l][f] for p in P) for f in F) for l in L) + quicksum(quicksum(quicksum(tonelada_transportada.sum(p,f,j) * ct[p][f][j] for p in P) for f in F) for j in J)
    modelo.setObjective(obj, GRB.MINIMIZE)
    
    modelo.setParam('TimeLimit', 1800)
        
    for p in P:
        for j in J:
            modelo.addConstr(quicksum(tonelada_transportada.sum(p, f, j) for f in F) == D[j][p])
            
    for p in P:
        for f in F:
            modelo.addConstr(quicksum(tonelada_transportada.sum(p, f, j) for j in J) == quicksum(tonelada_produzida.sum(p, l, f) for l in L))
    
    for f in F:
        for m in M:
            modelo.addConstr(quicksum(quicksum(tonelada_produzida.sum(p, l, f) * r[m][p][l] for p in P) for l in L) <= R[m][f])
            
    for f in F:
        for l in L:
            modelo.addConstr(quicksum(tonelada_produzida.sum(p, l, f) for p in P) <= C[l][f])
    
    modelo.update()
    modelo.write(f'atividade1-{len(J)}_clientes.lp')
    modelo.optimize()
    
    def imprimir_quantidades(custo, status):
        solucao = {
            'custo': custo,
            'status': str(status),
            'tonelada_produzida': [],
            'tonelada_transportada': []
        }
        
        for p in P:
            solucao['tonelada_produzida'].append([])
            for l in L:
                solucao['tonelada_produzida'][p].append([])
                for f in F:
                    solucao['tonelada_produzida'][p][l].append(tonelada_produzida[p, l, f].X)
                    
        for p in P:
            solucao['tonelada_transportada'].append([])
            for f in F:
                solucao['tonelada_transportada'][p].append([])
                for j in J:
                    solucao['tonelada_transportada'][p][f].append(tonelada_transportada[p, f, j].X)
                    
        imprimir_json(f'solucao-{len_J}_clientes.json', solucao)
    
    
    print()
    if modelo.status == GRB.Status.OPTIMAL:
        if imprimir_resultado:
            imprimir_quantidades(modelo.objVal, modelo.Status)
        print('Solução Ótima Encontrada!')
        print(f'Custo: {modelo.objVal}')
    elif modelo.status == GRB.Status.TIME_LIMIT:
        if imprimir_resultado:
            imprimir_quantidades(modelo.objVal, modelo.Status)
        print('Não foi possível encontrar uma solução ótima. Tempo excedido.')
        print('Solução: ')
        print(f'Custo: {modelo.objVal}')
    elif modelo.status == GRB.Status.INFEASIBLE:
        print('Não foi possível encontrar uma solução. Inviável.')
    else:
        print(f'Status do modelo não interpretado: {modelo.Status}')


In [4]:
#
# Instâncias (dados utilizados, fixos enquanto o notebook está aberto)
#

for len_J in [100 * x for x in inclrange(1, 10)]:
    print(f'Produzindo dados para {len_J} clientes...')
        
    J = [x for x in range(0, len_J)]  # conjunto de clientes
    # Número de...
    len_F = rand(len(J), 2 * len(J))
    F = [x for x in range(0, len_F)]  # fábricas
    
    len_L = rand(5, 10)  
    L = [x for x in range(0, len_L)]  # máquinas
    
    len_M = rand(5, 10)  
    M = [x for x in range(0, len_M)]  # matérias primas
    
    len_P = rand(5, 10)
    P = [x for x in range(0, len_P)]  # produtos
    
    D = []  # D_j,p: demanda de cliente j pelo produto p
    for j in J:
        D.append([])
        for p in P:
            D[j].append([])
            D[j][p] = rand(10, 20)
    
    r = []  # r_m,p,l: quantidade de matéria prima m para produzir produto p na máquina l
    for m in M:
        r.append([])
        for p in P:
            r[m].append([])
            for l in L:
                r[m][p].append([])
                r[m][p][l] = rand(1, 5)
                
    R = []  # R_m,f: quantidade de matéria prima m disponível na fábrica f
    for m in M:
        R.append([])
        for f in F:
            R[m].append([])
            R[m][f] = rand(800, 1000)
            
    C = []  # C_l,f: capacidade de produção da máquina l na fábrica f
    for l in L:
        C.append([])
        for f in F:
            C[l].append([])
            C[l][f] = rand(80, 100)
            
    cp = []  # p_p,l,f -> cp_p,l,f: custo de produção do produto p na máquina l na fábrica f
    for p in P:
        cp.append([])
        for l in L:
            cp[p].append([])
            for f in F:
                cp[p][l].append([])
                cp[p][l][f] = rand(10, 100)
    
    ct = []  # t_p,f,j -> ct_p,f,j: custo de transporte do produto p da fábrica f ao consumidor j
    for p in P:
        ct.append([])
        for f in F:
            ct[p].append([])
            for j in J:
                ct[p][f].append([])
                ct[p][f][j] = rand(10, 20)
    
    instancia = {
        'J': J,
        'F': F,
        'L': L,
        'M': M,
        'P': P,
        'D': D,
        'r': r,
        'R': R,
        'C': C,
        'cp': cp,
        'ct': ct
    }
    
    if imprimir_dados:
        imprimir_json(f'instancia-{len_J}_clientes.json', instancia)
    print(f'Solucionando para {len_J} clientes...')
    solucionar(**instancia)
    print('\n\n\n')

Produzindo dados para 100 clientes...
Solucionando para 100 clientes...
Set parameter Username
Academic license - for non-commercial use only - expires 2022-05-27
Set parameter TimeLimit to value 1800
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 4300 rows, 136081 columns and 322560 nonzeros
Model fingerprint: 0xd828356f
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [1e+01, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 1e+03]

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve removed 0 rows and 1 columns
Presolve time: 0.11s
Presolved: 4300 rows, 136080 columns, 322560 nonzeros

Ordering time: 0.10s

Barrier statistics:
 AA' NZ     : 1.514e+05
 Factor NZ  : 9.098e+05 (roughly 60 MB of memory)
 Factor Ops : 4.674e+08 (less than 1 second per iteration)
 Threads    : 6

Ba

Barrier solve interrupted - model solved by another algorithm


Solved with primal simplex
Solved in 1073334 iterations and 43.04 seconds (29.45 work units)
Optimal objective  1.278210877e+06

Solução Ótima Encontrada!
Custo: 1278210.8772727272




Produzindo dados para 700 clientes...
Solucionando para 700 clientes...
Set parameter TimeLimit to value 1800
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 28592 rows, 5433777 columns and 11350384 nonzeros
Model fingerprint: 0xfbae2d8e
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [1e+01, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 1e+03]

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve removed 0 rows and 1 columns (presolve time = 11s) ...
Presolve removed 0 rows and 1 columns
Presolve time: 14.91s
Presolved: 28592 rows,